# **MNIST**

Dataset MNIST adalah standar dalam *Machine Learning* untuk tugas klasifikasi. Scikit-Learn menyediakan fungsi untuk mengunduh dataset populer ini.
Dataset yang dimuat oleh Scikit-Learn umumnya memiliki struktur kamus dengan kunci seperti:
* `DESCR`: Deskripsi dataset.
* `data`: Array dengan satu baris per instansi dan satu kolom per fitur.
* `target`: Array dengan label.

Contoh pengambilan dataset MNIST:
```python
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
# mnist.keys() akan menampilkan dict_keys(['data', 'target', 'feature_names', 'DESCR', 'details', 'categories', 'url'])
```
Dataset ini berisi 70.000 gambar, masing-masing dengan 784 fitur. Ini karena setiap gambar berukuran $28 \times 28$ piksel, dan setiap fitur merepresentasikan intensitas piksel (0 untuk putih, 255 untuk hitam).
Contoh menampilkan salah satu digit:
```python
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

X, y = mnist["data"], mnist["target"]
# X.shape adalah (70000, 784), y.shape adalah (70000,)
some_digit = X.iloc[0] # Menggunakan .iloc karena X adalah DataFrame
some_digit_image = some_digit.values.reshape(28, 28) # .values untuk mengubah Series ke array numpy

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()
# y[0] akan menampilkan '5'
```
Label dalam dataset ini awalnya berupa *string*, sehingga perlu diubah ke integer karena sebagian besar algoritma ML mengharapkan angka.
```python
y = y.astype(np.uint8)
```
Dataset MNIST sudah dibagi menjadi *training set* (60.000 gambar pertama) dan *test set* (10.000 gambar terakhir). *Training set* sudah diacak (shuffled), yang baik untuk memastikan *cross-validation folds* serupa dan algoritma tidak terpengaruh oleh urutan instansi pelatihan.
```python
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
```

**Melatih *Binary Classifier***
Untuk menyederhanakan masalah, awalnya kita hanya akan mencoba mengidentifikasi satu digit, misalnya angka 5 (sebagai "detektor-5"). Ini adalah contoh *binary classifier* yang membedakan dua kelas: 5 dan bukan-5.
Target vektor untuk tugas ini dibuat:
```python
y_train_5 = (y_train == 5) # True untuk semua 5, False untuk digit lain
y_test_5 = (y_test == 5)
```
`SGDClassifier` (Stochastic Gradient Descent) dari Scikit-Learn adalah pilihan yang baik untuk memulai karena efisien dalam menangani dataset sangat besar, sebagian karena ia memproses instansi pelatihan secara independen.
Melatih `SGDClassifier`:
```python
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42) # random_state diatur untuk hasil yang dapat direproduksi
sgd_clf.fit(X_train, y_train_5)
```
Contoh prediksi:
```python
# sgd_clf.predict([some_digit]) # Akan menampilkan array([ True])
```

**Ukuran Performa (*Performance Measures*)**
Mengevaluasi *classifier* seringkali lebih rumit daripada *regressor*.

* **Mengukur Akurasi Menggunakan *Cross-Validation***:
    Seperti di Bab 2, *cross-validation* adalah cara yang baik untuk mengevaluasi model.
    Implementasi *cross-validation* secara manual:
    ```python
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone

    skfolds = StratifiedKFold(n_splits=3, random_state=42) # StratifiedKFold melakukan stratified sampling

    for train_index, test_index in skfolds.split(X_train, y_train_5):
        clone_clf = clone(sgd_clf)
        X_train_folds = X_train.iloc[train_index]
        y_train_folds = y_train_5.iloc[train_index]
        X_test_fold = X_train.iloc[test_index]
        y_test_fold = y_train_5.iloc[test_index]

        clone_clf.fit(X_train_folds, y_train_folds)
        y_pred = clone_clf.predict(X_test_fold)
        n_correct = sum(y_pred == y_test_fold)
        # print(n_correct / len(y_pred)) # Akan mencetak 0.9502, 0.96565, dan 0.96495
    ```
    Menggunakan `cross_val_score()`:
    ```python
    from sklearn.model_selection import cross_val_score

    # cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy") # Akan menghasilkan array sekitar [0.96355, 0.93795, 0.95615]
    ```
    Akurasi di atas 93% terlihat bagus, namun ini bisa menyesatkan untuk *skewed datasets* (ketika satu kelas jauh lebih sering daripada yang lain). Contohnya, *classifier* "Never5Classifier" yang selalu memprediksi "bukan-5" akan memiliki akurasi di atas 90% karena hanya sekitar 10% gambar yang merupakan angka 5. Ini menunjukkan mengapa akurasi bukan metrik performa yang utama untuk *classifier*.

* ***Confusion Matrix***:
    Cara yang lebih baik untuk mengevaluasi *classifier* adalah dengan melihat *confusion matrix*. Ini menghitung berapa kali instansi kelas A diklasifikasikan sebagai kelas B.
    Untuk menghitung *confusion matrix*, pertama-tama kita perlu serangkaian prediksi. Menggunakan `cross_val_predict()` akan memberikan prediksi "bersih" untuk setiap instansi di *training set*.
    ```python
    from sklearn.model_selection import cross_val_predict
    from sklearn.metrics import confusion_matrix

    y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
    # confusion_matrix(y_train_5, y_train_pred) # Akan menghasilkan array seperti [[53057, 1522], [1325, 4096]]
    ```
    Setiap baris dalam *confusion matrix* merepresentasikan kelas aktual, dan setiap kolom merepresentasikan kelas yang diprediksi.
    * **True Negatives (TN):** Gambar bukan-5 yang diklasifikasikan dengan benar sebagai bukan-5.
    * **False Positives (FP):** Gambar bukan-5 yang salah diklasifikasikan sebagai 5.
    * **False Negatives (FN):** Gambar 5 yang salah diklasifikasikan sebagai bukan-5.
    * **True Positives (TP):** Gambar 5 yang diklasifikasikan dengan benar sebagai 5.
    *Classifier* yang sempurna hanya akan memiliki TP dan TN, dengan nilai bukan-nol hanya pada diagonal utamanya.

    * **Precision (*Presisi*):** Akurasi prediksi positif.
        $$\text{precision} = \frac{TP}{TP+FP}$$
        Precision tinggi berarti ketika *classifier* mengklaim sesuatu adalah kelas positif, itu sebagian besar benar.
    * **Recall (*Sensitivitas* atau *True Positive Rate - TPR*):** Rasio instansi positif yang terdeteksi dengan benar oleh *classifier*.
        $$\text{recall} = \frac{TP}{TP+FN}$$
        Recall tinggi berarti *classifier* menemukan sebagian besar instansi positif.

    Contoh perhitungan *precision* dan *recall* untuk *detector*-5:
    ```python
    from sklearn.metrics import precision_score, recall_score

    # precision_score(y_train_5, y_train_pred) # Sekitar 0.72908
    # recall_score(y_train_5, y_train_pred) # Sekitar 0.75558
    ```
    Ini menunjukkan bahwa ketika *classifier* mengklaim sebuah gambar adalah 5, itu benar hanya 72.9% dari waktu, dan hanya mendeteksi 75.6% dari angka 5 yang sebenarnya.

    * ***F1 Score***:
        Metrik gabungan *precision* dan *recall* yang berguna untuk membandingkan *classifier*. Ini adalah *harmonic mean* dari *precision* dan *recall*, yang memberikan bobot lebih pada nilai yang rendah, sehingga *classifier* akan mendapatkan *F1 score* tinggi hanya jika *precision* dan *recall* keduanya tinggi.
        $$F_{1}=\frac{2}{\frac{1}{precision}+\frac{1}{recall}}=2\times\frac{precision\times recall}{Precision+recall}=\frac{TP}{TP+\frac{FN+FP}{2}}$$
    Contoh perhitungan *F1 score*:
    ```python
    from sklearn.metrics import f1_score

    # f1_score(y_train_5, y_train_pred) # Sekitar 0.74209
    ```
    *F1 score* lebih menyukai *classifier* dengan *precision* dan *recall* yang serupa.

* **Precision/Recall Trade-off**:
    Meningkatkan *precision* akan mengurangi *recall*, dan sebaliknya. Ini disebut *precision/recall trade-off*. `SGDClassifier` membuat keputusan klasifikasi berdasarkan fungsi keputusan. Jika skor dari fungsi keputusan lebih besar dari ambang batas (*threshold*), instansi ditetapkan ke kelas positif; jika tidak, ke kelas negatif. Menaikkan ambang batas akan meningkatkan *precision* (mengurangi *false positives*) tetapi menurunkan *recall* (meningkatkan *false negatives*). Sebaliknya, menurunkan ambang batas akan meningkatkan *recall* dan mengurangi *precision*.

    Scikit-Learn memungkinkan akses ke skor keputusan melalui metode `decision_function()`:
    ```python
    # y_scores = sgd_clf.decision_function([some_digit]) # Akan menampilkan array([2412.53175101])
    # threshold = 0
    # y_some_digit_pred = (y_scores > threshold) # Akan menghasilkan array([ True])
    # threshold = 8000
    # y_some_digit_pred = (y_scores > threshold) # Akan menghasilkan array([False])
    ```
    Untuk memutuskan ambang batas mana yang akan digunakan, kita bisa mendapatkan skor keputusan untuk semua instansi di *training set* menggunakan `cross_val_predict()` dengan `method="decision_function"`.
    ```python
    y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                                 method="decision_function")
    ```
    Kemudian, `precision_recall_curve()` dapat digunakan untuk menghitung *precision* dan *recall* untuk semua kemungkinan ambang batas.
    ```python
    from sklearn.metrics import precision_recall_curve

    precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)
    ```
    Plot *precision* dan *recall* sebagai fungsi dari nilai ambang batas dapat membantu memilih *trade-off* yang sesuai.
    ```python
    def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
        plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
        plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
        # Tambahkan kode untuk label dan grid jika perlu
        plt.xlabel("Threshold")
        plt.legend(loc="center left")
        plt.grid(True)

    # plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
    # plt.show()
    ```
    *Precision* mulai menurun tajam sekitar *recall* 80%. Pilihan *trade-off* tergantung pada proyek.
    Misalnya, untuk mencapai *precision* 90%:
    ```python
    threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)] # Sekitar ~7816
    y_train_pred_90 = (y_scores >= threshold_90_precision)
    # precision_score(y_train_5, y_train_pred_90) # Sekitar 0.90003
    # recall_score(y_train_5, y_train_pred_90) # Sekitar 0.43681
    ```
    Menciptakan *classifier* dengan *precision* tinggi cukup mudah dengan mengatur ambang batas yang tinggi, tetapi *recall* yang terlalu rendah membuatnya tidak terlalu berguna.

* **The ROC Curve**:
    *Receiver Operating Characteristic (ROC) curve* adalah alat umum lain untuk *binary classifier*. Kurva ini memplot *true positive rate* (recall) terhadap *false positive rate* (FPR). FPR adalah rasio instansi negatif yang salah diklasifikasikan sebagai positif, dan sama dengan 1 - *true negative rate* (specificity). Jadi, kurva ROC memplot *sensitivitas* versus 1 - *spesifisitas*.
    Untuk memplot kurva ROC, gunakan fungsi `roc_curve()` untuk menghitung TPR dan FPR untuk berbagai nilai ambang batas.
    ```python
    from sklearn.metrics import roc_curve

    fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)
    ```
    Plot FPR terhadap TPR:
    ```python
    def plot_roc_curve(fpr, tpr, label=None):
        plt.plot(fpr, tpr, linewidth=2, label=label)
        plt.plot([0, 1], [0, 1], 'k--') # Garis diagonal putus-putus untuk classifier acak
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate (Recall)")
        plt.grid(True)

    # plot_roc_curve(fpr, tpr)
    # plt.show()
    ```
    Ada *trade-off* lain: semakin tinggi *recall* (TPR), semakin banyak *false positives* (FPR) yang dihasilkan *classifier*. *Classifier* yang baik akan menjauh dari garis diagonal (menuju sudut kiri atas).
    * **Area Under the Curve (AUC)**:
        Salah satu cara membandingkan *classifier* adalah dengan mengukur *Area Under the Curve* (AUC). *Classifier* sempurna memiliki ROC AUC 1, sedangkan *classifier* acak murni memiliki ROC AUC 0.5.
    ```python
    from sklearn.metrics import roc_auc_score

    # roc_auc_score(y_train_5, y_scores) # Sekitar 0.96117
    ```
    Sebagai aturan umum, pilih kurva PR (*precision/recall*) jika kelas positif jarang atau jika *false positives* lebih penting daripada *false negatives*. Jika tidak, gunakan kurva ROC.

    Membandingkan `RandomForestClassifier` dengan `SGDClassifier`:
    `RandomForestClassifier` tidak memiliki metode `decision_function()`, tetapi memiliki metode `predict_proba()` yang mengembalikan probabilitas bahwa suatu instansi termasuk dalam kelas tertentu.
    ```python
    from sklearn.ensemble import RandomForestClassifier

    forest_clf = RandomForestClassifier(random_state=42)
    y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                         method="predict_proba")
    y_scores_forest = y_probas_forest[:, 1] # Skor probabilitas kelas positif
    fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

    # plt.plot(fpr, tpr, "b:", label="SGD")
    # plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
    # plt.legend(loc="lower right")
    # plt.show()
    ```
    Kurva ROC `RandomForestClassifier` terlihat jauh lebih baik, lebih dekat ke sudut kiri atas, dan memiliki skor ROC AUC yang jauh lebih baik.
    ```python
    # roc_auc_score(y_train_5, y_scores_forest) # Sekitar 0.99834
    ```
    *Precision* dan *recall* untuk `RandomForestClassifier` ini adalah sekitar 99.0% *precision* dan 86.6% *recall*.

**Klasifikasi Multikelas (*Multiclass Classification*)**
*Multiclass classifier* membedakan lebih dari dua kelas. Beberapa algoritma (seperti *SGD classifiers*, *Random Forest classifiers*, *naive Bayes classifiers*) dapat menangani banyak kelas secara native. Algoritma lain (seperti *Logistic Regression* atau *Support Vector Machine classifiers*) adalah *binary classifier* ketat.
Ada beberapa strategi untuk melakukan klasifikasi multikelas dengan *binary classifier*:
* ***One-versus-the-rest (OvR)*** atau ***One-versus-all***: Melatih N *binary classifier*, satu untuk setiap kelas. Saat mengklasifikasikan gambar, ambil skor keputusan dari setiap *classifier* dan pilih kelas dengan skor tertinggi.
* ***One-versus-one (OvO)***: Melatih *binary classifier* untuk setiap pasangan digit. Jika ada N kelas, dibutuhkan $N \times (N-1)/2$ *classifier*. Keuntungan OvO adalah setiap *classifier* hanya perlu dilatih pada bagian *training set* untuk dua kelas yang harus dibedakannya. OvO lebih disukai untuk algoritma yang tidak berskala baik dengan ukuran *training set* (misalnya *Support Vector Machine classifiers*).

Scikit-Learn secara otomatis mendeteksi ketika Anda mencoba menggunakan algoritma klasifikasi biner untuk tugas klasifikasi multikelas dan secara otomatis menjalankan OvR atau OvO, tergantung pada algoritmanya.
Contoh dengan `SVC` (Support Vector Machine classifier):
```python
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train) # Menggunakan y_train asli (0-9)
# svm_clf.predict([some_digit]) # Akan menampilkan array([5])
```
Di balik layar, Scikit-Learn menggunakan strategi OvO (melatih 45 *binary classifier*). Metode `decision_function()` akan mengembalikan 10 skor per instansi (satu skor per kelas).
```python
# some_digit_scores = svm_clf.decision_function([some_digit])
# np.argmax(some_digit_scores) # Akan menampilkan 5
# svm_clf.classes_ # Menampilkan array ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
```
Jika ingin memaksa Scikit-Learn menggunakan OvR atau OvO, gunakan kelas `OneVsOneClassifier` atau `OneVsRestClassifier`.
```python
from sklearn.multiclass import OneVsRestClassifier

ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, y_train)
# ovr_clf.predict([some_digit]) # Akan menampilkan array([5])
# len(ovr_clf.estimators_) # Akan menampilkan 10
```
Melatih `SGDClassifier` (atau `RandomForestClassifier`) untuk multikelas lebih mudah karena mereka dapat mengklasifikasikan instansi ke dalam banyak kelas secara langsung.
```python
sgd_clf.fit(X_train, y_train)
# sgd_clf.predict([some_digit]) # Akan menampilkan array([5])
# sgd_clf.decision_function([some_digit]) # Akan menampilkan 10 skor untuk setiap kelas
```
Mengevaluasi akurasi *SGDClassifier* multikelas menggunakan *cross-validation*:
```python
# cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy") # Sekitar [0.84898, 0.87129, 0.86988]
```
Skala input dapat meningkatkan akurasi:
```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy") # Sekitar [0.89707, 0.89609, 0.90693]
```

**Analisis Kesalahan (*Error Analysis*)**
Untuk meningkatkan *classifier*, analisis jenis kesalahan yang dibuatnya.
Lihat *confusion matrix* dari *classifier* multikelas:
```python
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
# conf_mx # Akan menampilkan matriks dengan banyak angka
```
Representasi visual menggunakan `matshow()` dari Matplotlib lebih mudah dilihat:
```python
# plt.matshow(conf_mx, cmap=plt.cm.gray)
# plt.show()
```
*Confusion matrix* ini terlihat cukup baik karena sebagian besar gambar berada pada diagonal utama (diklasifikasikan dengan benar). Angka 5 terlihat sedikit lebih gelap, menunjukkan ada lebih sedikit gambar 5 atau *classifier* tidak berkinerja sebaik pada 5.
Untuk fokus pada kesalahan, bagi setiap nilai dalam *confusion matrix* dengan jumlah gambar di kelas yang sesuai untuk membandingkan tingkat kesalahan. Kemudian isi diagonal dengan nol.
```python
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)
# plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
# plt.show()
```
Dari visualisasi ini, terlihat jelas jenis kesalahan yang dibuat *classifier*. Kolom untuk kelas 8 cukup cerah, yang berarti banyak gambar salah diklasifikasikan sebagai 8. Namun, baris untuk kelas 8 tidak terlalu buruk, artinya 8 yang sebenarnya secara umum diklasifikasikan dengan benar. *Confusion matrix* tidak harus simetris. Juga terlihat bahwa 3 dan 5 seringkali saling tertukar.

Menganalisis *confusion matrix* memberikan wawasan untuk meningkatkan *classifier*. Misalnya, untuk mengurangi kesalahan pada angka 8, bisa dengan mengumpulkan lebih banyak data pelatihan untuk digit yang mirip 8 tetapi bukan 8, atau menciptakan fitur baru seperti menghitung jumlah lingkaran tertutup. Pra-pemrosesan gambar juga bisa membantu.

Menganalisis kesalahan individual juga bermanfaat, meskipun lebih sulit. Misalnya, 3 dan 5 sering salah diklasifikasikan oleh `SGDClassifier` (model linear) karena mereka hanya berbeda beberapa piksel. *Classifier* ini sensitif terhadap pergeseran dan rotasi gambar. Pra-pemrosesan gambar untuk memastikan mereka terpusat dan tidak terlalu berputar dapat mengurangi kebingungan 3/5.

**Klasifikasi Multilabel (*Multilabel Classification*)**
Dalam *multilabel classification*, *classifier* dapat mengeluarkan beberapa kelas untuk setiap instansi. Contohnya, sistem pengenalan wajah yang mengenali beberapa orang dalam satu gambar akan melampirkan satu tag per orang yang dikenalinya.
Contoh sederhana dengan MNIST:
```python
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= 7) # Digit besar (7, 8, atau 9)
y_train_odd = (y_train % 2 == 1) # Digit ganjil
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier() # Mendukung multilabel classification
knn_clf.fit(X_train, y_multilabel)
```
Prediksi untuk digit 5:
```python
# knn_clf.predict([some_digit]) # Akan menampilkan array([[False, True]])
```
Ini benar karena 5 bukan digit besar (False) dan ganjil (True).
Untuk mengevaluasi *multilabel classifier*, salah satu pendekatan adalah mengukur *F1 score* untuk setiap label dan menghitung skor rata-rata.
```python
# y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
# f1_score(y_multilabel, y_train_knn_pred, average="macro") # Sekitar 0.97641
```
`average="macro"` mengasumsikan semua label sama penting. Untuk memberikan bobot berbeda (misalnya berdasarkan *support* label), gunakan `average="weighted"`.

**Klasifikasi Multioutput (*Multioutput Classification*)**
*Multioutput-multiclass classification* (atau *multioutput classification*) adalah generalisasi dari *multilabel classification* di mana setiap label bisa berupa multikelas (memiliki lebih dari dua nilai).
Contohnya adalah sistem penghilang *noise* dari gambar digit. Inputnya adalah gambar digit ber-*noise*, dan outputnya adalah gambar digit bersih, direpresentasikan sebagai array intensitas piksel. Output *classifier* ini adalah multilabel (satu label per piksel) dan setiap label dapat memiliki banyak nilai (intensitas piksel 0-255). Ini adalah contoh sistem klasifikasi *multioutput*.
Membangun *training set* dan *test set* dengan menambahkan *noise* pada gambar MNIST:
```python
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise

noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise

y_train_mod = X_train # Targetnya adalah gambar asli
y_test_mod = X_test
```
Melatih *classifier* untuk membersihkan gambar ini:
```python
# knn_clf.fit(X_train_mod, y_train_mod)
# clean_digit = knn_clf.predict([X_test_mod.iloc[some_index]]) # Menggunakan .iloc karena X_test_mod adalah DataFrame
# plot_digit(clean_digit) # Diharapkan menghasilkan gambar digit yang bersih
```
Ini mengakhiri pembahasan klasifikasi, mencakup pemilihan metrik, *precision/recall trade-off*, perbandingan *classifier*, dan membangun sistem klasifikasi yang baik untuk berbagai tugas.